<a href="https://colab.research.google.com/github/NadirYasar/Ecommerce_Customers/blob/main/Ecommerce_Customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install findspark

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark= SparkSession.builder.appName('Linear Regression').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
training =spark.read.csv('/content/Ecommerce_Customers.csv',inferSchema=True,header=True)
training.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [ ]:
training.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
training.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             NULL|                NULL|       NULL| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             NULL|                NULL|       NULL|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

categorical_columns = ['Email','Address','Avatar']

indexers = [
    StringIndexer(inputCol = col, outputCol = col + "_cat") for col in categorical_columns
]

pipeline  = Pipeline(stages=indexers)
training_df = pipeline.fit(training).transform(training)
training_df.select('Email','Address','Avatar','Email_cat','Address_cat','Avatar_cat')



DataFrame[Email: string, Address: string, Avatar: string, Email_cat: double, Address_cat: double, Avatar_cat: double]

In [ ]:
training_df.head(5)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, Email_cat=342.0, Address_cat=381.0, Avatar_cat=96.0),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264, Email_cat=190.0, Address_cat=226.0, Avatar_cat=26.0),
 Row(Email='pallen@yahoo.com', Address='24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564', Avatar='Bisque', Avg Session Length=33.000914755642675, Time on App=11.330278057777512, Time on Website=37.110597442120856, Length of Membership=4.104543202376424, Yearly Amount Spent=487.54750486747207, Em

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
training_df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'Email_cat',
 'Address_cat',
 'Avatar_cat']

In [ ]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length',
               'Time on App',
               'Time on Website',
               'Length of Membership',
               'Yearly Amount Spent',
               'Email_cat',
               'Address_cat',
               'Avatar_cat'],
    outputCol = "features"
)

In [ ]:
output = assembler.transform(training_df)

In [ ]:
output.select("features", "Yearly Amount Spent").show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [ ]:
final_data = output.select("features", "Yearly Amount Spent")

In [ ]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [ ]:
lrModel = lr.fit(train_data)

In [ ]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-4.210586963692817e-12,-7.184990139238675e-12,4.64105064743974e-12,-1.0305942178568944e-11,1.0000000000001794,1.7422056307499582e-15,2.0603444045048283e-15,-7.659638650800935e-15] Intercept: 0.0


In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 5.39451365637087e-12
MSE: 2.910077758877181e-23
R2: 1.0


In [ ]:
from pyspark.sql.functions import corr


In [ ]:
training.select(corr('Yearly Amount Spent','Length of Membership')).show()

+-----------------------------------------------+
|corr(Yearly Amount Spent, Length of Membership)|
+-----------------------------------------------+
|                             0.8090835682475807|
+-----------------------------------------------+



In [ ]:
training.select(corr('Yearly Amount Spent','Avg Session Length')).show()

+---------------------------------------------+
|corr(Yearly Amount Spent, Avg Session Length)|
+---------------------------------------------+
|                          0.35508829470764314|
+---------------------------------------------+

